# Gridworld Speedup

In [199]:
import random 
import math
import matplotlib.pyplot as plt

In [200]:
# Episodes 
episodes = 170

# Start
start = (4, 0)

# Goal
goal = (4, 6)

# alpha 
alpha = 0.1

# n-step
n = 16

# gamma 
gamma = 0.9


In [201]:
def create():
    """
    Create and return Q and arbitrary policy
    """
    
    # Create length and width to iterate grid
    length = 10; width = 8

    # Set of equiprobable actions
    actions = [(-1, 0), (1, 0), (0, 1), (0, -1)]
    # Policy
    policy = {}; Q = {}

    # Create states for policy
    for i in range(length*width):

        # Create row index
        row = i // length

        # Create column index
        column = i % length

        # Create policy
        policy[(row, column)] = actions
        
        # Create Q(s,a)
        Q[(row, column)] = [0 for action in actions]
        
    # Q and policy 
    return Q, policy

In [202]:
def pi(Q, policy, state):
    """
    Target policy probability greedy
    """
    
    # Epsilon
    e = 0.1
    
    # Create action probability
    Pr = {action: e/len(policy[state]) for action in policy[state]}
    
    # Exploitation Arg max a
    argmax = Q[state].index(max(Q[state]))
    
    # greedy action
    action = policy[state][argmax]
    
    # Set max probability
    Pr[action] += (1-e)/len(policy[state])
    
    return Pr

In [203]:
def b(Q, policy,  state):
    """
    Behavior policy probability e-greedy
    """
    
    # Set epsilon
    e = 0.1
    
    # Create action probability
    Pr = {action: e/len(policy[state]) for action in policy[state]}
    
    # Exploitation Arg max a
    argmax = Q[state].index(max(Q[state]))
    
    # e-greedy action
    action = policy[state][argmax]
    
    # Set max probability
    Pr[action] = (1 - e)/len(policy[state])
    
    return Pr

In [204]:
def pi_v(V, policy, state):
    """
    Target policy probability greedy w.r.t. V
    """
    
    # Epsilon
    e = 0.1
    
    # Create action probability
    Pr = {action: e/len(policy[state]) for action in policy[state]}
    
    # list of values
    max_v = []
    
    # Exploitation max V(s')
    for action in policy[state]:
        
        # create neighbor values
        if (state[0] + action[0], state[1] + action[1]) in V:
            max_v += [V[(state[0] + action[0], state[1] + action[1])]] 
        
        # keep current state values for out of bounds
        else:
            
            max_v += [V[state]]
        
    # Arg max a
    argmax = max_v.index(max(max_v))
    
    # e-greedy action
    action = policy[state][argmax]
    
    # Set max probability
    Pr[action] += (1-e)/len(policy[state])
    
    return Pr

In [205]:
def b_v(V, policy,  state):
    """
    Behavior policy probability e-greedy w.r.t. V
    """
    
    # Set epsilon
    e = 0.1
    
    # Create action probability
    Pr = {action: e/len(policy[state]) for action in policy[state]}
    
    # list of values
    max_v = []
    
    # Exploitation max V(s')
    for action in policy[state]:
        
        # create neighbor values
        if (state[0] + action[0], state[1] + action[1]) in V:
            max_v += [V[(state[0] + action[0], state[1] + action[1])]] 
        
        # keep current state values for out of bounds
        else:
            
            max_v += [V[state]]
        
    # Arg max a
    argmax = max_v.index(max(max_v))
    
    # e-greedy action
    action = policy[state][argmax]
    
    # Set max probability
    Pr[action] = (1 - e)/len(policy[state])
    
    return Pr

In [206]:
def boundary(state):
    """
    Returns if agent is in bounds
    """
    # row boundary
    row_boundary = (state[0] < 0 or state[0] > 7) 
        
    # column boundary
    column_boundary = (state[1] < 0 or state[1] > 9) 
    
    # if not in bounds
    return row_boundary or column_boundary

In [207]:
def e_greedy(Q, policy, state, goal):
    """
    Return an action based on e-greedy policy
    """

    # Exploitation Arg max a
    argmax = Q[state].index(max(Q[state]))
    
    # e-greedy action
    action = policy[state][argmax]
    
    # Exploration
    if random.random() < 0.1:
        
        # Explorative action
        action = random.choice(policy[state])
    
    # e-greedy action
    return action

In [208]:
def greedy(Q, policy, state):
    """
    Return an action based on greedy policy
    """

    # Exploitation Arg max a
    argmax = Q[state].index(max(Q[state]))

    # greedy action
    return policy[state][argmax]

In [209]:
def e_greedy_v(V, policy, state, goal):
    """
    Return an action based on e-greedy policy
    """

    # list of values
    max_v = []
    
    # Exploitation max V(s')
    for action in policy[state]:
        
        # create neighbor values
        if (state[0] + action[0], state[1] + action[1]) in V:
            max_v += [V[(state[0] + action[0], state[1] + action[1])]] 
        
        # keep current state values for out of bounds
        else:
            
            max_v += [V[state]]
        
    # Arg max a
    argmax = max_v.index(max(max_v))
    
    # e-greedy action
    action = policy[state][argmax]
    
    # Exploration
    if random.random() < 0.1:
        
        # Explorative action
        action = random.choice(policy[state])
    
    # e-greedy action
    return action

In [210]:
def move(state, action, goal):
    """
    Return environment obervation
    """
    
    # S' after a
    s_prime = (state[0] + action[0], state[1] + action[1])
    
    # r'
    reward = 0
    
    # Check goal reached
    if s_prime == goal:
        
        # S', R = 1 on termination
        return s_prime, 1
    
    # check if out of bounds
    if boundary(s_prime):
        
        # Remain in state
        s_prime = state
        
        # consequence
        reward = -1
    
    # S', R'
    return s_prime, reward

In [211]:
def n_step_sarsa(alpha, n, episodes, start, goal):
    """
    n-step SARSA for estimating Q = q* or q_pi
    """
    
    # Initialize Q(s, a) arbitrarily, for all s of S, a of A
    # Initialize ⇡ to be "-greedy with respect to Q, or to a fixed given policy
    Q, policy = create()
    
    # All store and access operations (for St and Rt) can take their index mod n + 1
    store = []
    
    # Loop for each episode:
    for i in range(episodes):
        
        # Initialize S0 != terminal
        S = start
        
        # Select and store an action A0 = pi(·|S0)
        A = e_greedy(Q, policy, S, goal)
        
        # Store S0 != terminal
        store.append((S, A, 0))
        
        # T <-- inf
        T = float('inf');  t = tau = 0
        
        # Loop for t = 0, 1, 2,... : Until t' = T - 1
        while tau != T-1:

            # If t < T, then:
            if t < T:
                
                # Take action At
                S, R = move(S, A, goal)
                
                # Observe and store the next reward as Rt+1 and the next state as St+1
                store.append((S, A, R))
                
                # If St+1 is terminal, then T <-- t + 1
                if S == goal: T = t + 1
                    
                # Select and store an action A0 = pi(·|S0)
                else: A = e_greedy(Q, policy, S, goal)
                
            # t' <-- t - n +1 (t' is the time whose state’s estimate is being updated)
            tau = t - n + 1
            
            # If t' >= 0:
            if tau >= 0:
            
                # G <-- sum(i=t'+1 to min(t'+n,T)) gamma^i-t'-1*Ri
                G = sum([store[i][2] for i in range(tau + 1, min(tau + n, T)+1)])
                
                # If t'+n < T, then: G <-- G + gamma^n*Q(St', At')
                if tau+n < T: G = G + Q[store[tau+n][0]][policy[store[tau][0]].index(store[tau][1])]

                # Q(St', At') <-- Q(St', At') + a[G - Q(St', At')]
                Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] = Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] + alpha*(G - Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])])
            
            # Until t' = T - 1
            t += 1
            
    # Return q*
    return Q

In [212]:
def n_step_off_policy_sarsa(alpha, n, episodes, start, goal):
    """
    Off-policy n-step Sarsa for estimating Q = q* or q_p⇡
    """
    
    # Input: an arbitrary behavior policy b such that b(a|s) > 0, for all s of S, a of A
    # Initialize Q(s, a) arbitrarily, for all s of S, a of A
    # Initialize pi to be greedy with respect to Q, or as a fixed given policy
    Q, policy = create()
    
    # All store and access operations (for St, At, and Rt) can take their index mod n + 1
    store = []

    # Loop for each episode:
    for _ in range(episodes):
        
        # Initialize S0 != terminal
        S = start
        
        # Select an Action A0
        A = e_greedy(Q, policy, S, goal)
        
        # store S0 != terminal and A0 ⇠ b(·|S0)
        store.append((S, A, 0))
        
        # T <-- infinity
        T = float('inf');  t = tau = 0
        
        # Loop for t = 0, 1, 2,... : Until tau = T - 1
        while tau != T-1:
            
            # If t < T, then:
            if t < T:
                
                # Take action At Observe the next reward as Rt+1 and the next state as St+1
                S, R = move(S, A, goal)
                
                # store the next reward as Rt+1 and the next state as St+1
                store.append((S, A, R))
                
                # If St+1 is terminal, then: T <-- t + 1
                if S == goal: T = t + 1
                    
                # else: Select and store an action At+1 ⇠ b(·|St+1)
                else: A = e_greedy(Q, policy, S, goal)

            # tau <-- t - n +1
            tau = t - n + 1
            
            # If tau >=  0:
            if tau >= 0:
                
                # p <-- 1
                rho = 1
                
                # from i=tau+1 to min(tau+n,T-1)
                for i in range(tau + 1, min(tau + n, T-1)+1):
                    
                    # p <-- Projection of pi(Ai|Si) / b(Ai|Si)
                    rho *= (pi(Q, policy, store[i][0])[store[i][1]]/b(Q, policy, store[i][0])[store[i][1]])

                # G <-- Sum from i=tau+1 to min(tau+n,T) of gamma^i-tau-1 * Ri
                G = sum([store[i][2] for i in range(tau + 1, min(tau + n, T)+1)])
                
                # If tau + n < T, then: G <-- G + gamma^n*Q(Stau+n, Atau+n) 
                if tau + n < T: G = G + Q[store[tau+n][0]][policy[store[tau][0]].index(store[tau][1])]

                # Q(Stau, Atau) <-- Q(Stau , Atau) + a*p*[G - Q(Stau, Atau)]
                Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] = Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] + alpha*rho*(G - Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])]) 
            
            # Until t' = T - 1
            t += 1
            
    # Q = q*
    return Q

In [213]:
def n_step_per_decision(alpha, n, episodes, start, goal):
    """
    Off-policy n-step Sarsa per-decision importance sampling
    """
    
    # Input: policy
    V, policy = create()
    
    # Initialize V(s) arbitrarily, for all s of S
    V = V.fromkeys(V, 0)
    
    # All store and access operations (for St, At, and Rt) can take their index mod n + 1
    store = []

    # Loop for each episode:
    for _ in range(episodes):
        
        # Initialize S0 != terminal
        S = start
        
        # Select an Action A0
        A = e_greedy_v(V, policy, S, goal)
        
        # store S0 != terminal and A0 ⇠ b(·|S0)
        store.append((S, A, 0))
        
        # T <-- infinity
        T = float('inf');  t = tau = 0
        
        # Loop for t = 0, 1, 2,... : Until tau = T - 1
        while tau != T-1:
            
            # If t < T, then:
            if t < T:
                
                # Take action At Observe the next reward as Rt+1 and the next state as St+1
                S, R = move(S, A, goal)
                
                # store the next reward as Rt+1 and the next state as St+1
                store.append((S, A, R))
                
                # If St+1 is terminal, then: T <-- t + 1
                if S == goal: T = t + 1
                    
                # else: Select and store an action At+1 ⇠ b(·|St+1)
                else: A = e_greedy_v(V, policy, S, goal)

            # tau <-- t - n +1
            tau = t - n + 1
            
            # If tau >=  0:
            if tau >= 0:
                
                # p <-- 1
                rho = 1
                
                # from i=tau+1 to min(tau+n,T-1)
                for i in range(tau + 1, min(tau + n, T-1)+1):
                    
                    # p <-- Projection of pi(Ai|Si) / b(Ai|Si)
                    rho *= (pi_v(V, policy, store[i][0])[store[i][1]]/b_v(V, policy, store[i][0])[store[i][1]])

                # G <-- Sum from i=tau+1 to min(tau+n,T) of gamma^i-tau-1 * Ri
                G = sum([store[i][2] for i in range(tau + 1, min(tau + n, T)+1)])
                
                # If tau + n < T, then: G <-- G + gamma^n*Q(Stau+n, Atau+n) 
                if tau + n < T: G = rho*G + (1-rho)*V[store[tau+n][0]]

                # V(Stau) <-- V(Stau) + a*p*[G - V(Stau)]
                V[store[tau][0]] = V[store[tau][0]] + alpha*(G - V[store[tau][0]]) 
            
            # Until t' = T - 1
            t += 1
            
    # V = v*
    return V

In [214]:
def n_step_tree(alpha, n, episodes, start, goal):
    """
    n-step tree backup algorithm estimating Q = q* or q_p⇡
    """
    
    # Input: an arbitrary behavior policy b such that b(a|s) > 0, for all s of S, a of A
    # Initialize Q(s, a) arbitrarily, for all s of S, a of A
    # Initialize pi to be greedy with respect to Q, or as a fixed given policy
    Q, policy = create()
    
    # All store and access operations (for St, At, and Rt) can take their index mod n + 1
    store = []

    # Loop for each episode:
    for _ in range(episodes):
        
        # Initialize S0 != terminal
        S = start
        
        # Select an Action A0
        A = e_greedy(Q, policy, S, goal)
        
        # store S0 != terminal and A0 ⇠ b(·|S0)
        store.append((S, A, 0))
        
        # T <-- infinity
        T = float('inf');  t = tau = 0
        
        # Loop for t = 0, 1, 2,... : Until tau = T - 1
        while tau != T-1:
            
            # If t < T, then:
            if t < T:
                
                # Take action At Observe the next reward as Rt+1 and the next state as St+1
                S, R = move(S, A, goal)
                
                # store the next reward as Rt+1 and the next state as St+1
                store.append((S, A, R))
                
                # If St+1 is terminal, then: T <-- t + 1
                if S == goal: T = t + 1
                    
                # else: Select and store an action At+1 ⇠ b(·|St+1)
                else: A = e_greedy(Q, policy, S, goal)

            # tau <-- t - n +1
            tau = t - n + 1
            
            # If tau >=  0:
            if tau >= 0:
                
                # If t + 1 >= T: 
                if t+1 >= T: 
                    
                    # G <-- RT
                    G = store[T][2]
               
                # else: 
                else:
                    
                    # G <-- Rt+1 + gamma *SUM a pi(a|St+1)*Q(St+1,a)
                    G = store[t+1][2] + pi(Q, policy, store[t+1][0])[store[t+1][1]] * Q[store[t+1][0]][policy[store[t+1][0]].index(store[t+1][1])]
                    
                # Loop for k = min(t, T  1) down through tau + 1:
                for k in range(min(t+1, T), tau+1, 1):
                    
                    # if a == Ak
                    sum_a = 0
                    
                    # Sum if a != Ak 
                    if A != store[k][1]:
                    
                        sum_a += pi(Q, policy, store[k][0])[store[k][1]]*Q(store[k][0], store[k][1]) + pi(Q, policy, store[k][0])[store[k][1]]*Q(store[k][0], store[k][1]) 
                    
                    # G <-- Rk + Gammma*SUM a!=Ak pi(a|Sk)*Q(Sk, a) + gamma*pi(Ak|Sk)*G
                    G = store[k][2] + sum_a + pi(Q, policy, store[k][0])[store[k][1]]*G
                    
                # If tau + n < T, then: G <-- G + gamma^n*Q(Stau+n, Atau+n) 
                if tau + n < T: G = G + Q[store[tau+n][0]][policy[store[tau+n][0]].index(store[tau+n][1])]

                # Q(Stau, Atau) <-- Q(Stau , Atau) + a*p*[G - Q(Stau, Atau)]
                Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] = Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] + alpha*(G - Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])]) 
            
            # Until t' = T - 1
            t += 1
            
    # Q = q*
    return Q

In [215]:
def n_step_qsigma(alpha, gamma, n, episodes, start, goal):
    """
    Off-policy n-step Q(sigma) for estimating Q = q* or qp⇡
    """
    
    # Input: an arbitrary behavior policy b such that b(a|s) > 0, for all s of S, a of A
    # Initialize Q(s, a) arbitrarily, for all s of S, a of A
    # Initialize pi to be greedy with respect to Q, or as a fixed given policy
    Q, policy = create()
    
    # All store and access operations (for St, At, and Rt) can take their index mod n + 1
    store = []; rho = []; sigma = []; sampling = False; G = V = 0

    # Loop for each episode:
    for _ in range(episodes):
        
        # Initialize S0 != terminal
        S = start
        
        # Select an Action A0
        A = e_greedy(Q, policy, S, goal)
        
        # store S0 != terminal and A0 ⇠ b(·|S0)
        store.append((S, A, 0))
        
        # T <-- infinity
        T = float('inf');  t = tau = 0
        
        # Loop for t = 0, 1, 2,... : Until tau = T - 1
        while tau != T-1:
            
            # If t < T, then:
            if t < T:
                
                # Take action At Observe the next reward as Rt+1 and the next state as St+1
                S, R = move(S, A, goal)
                
                # store the next reward as Rt+1 and the next state as St+1
                store.append((S, A, R))
                
                # If St+1 is terminal, then: T <-- t + 1
                if S == goal: T = t + 1
                    
                # else: Select and store an action At+1 ⇠ b(·|St+1)
                else: 
                    
                    # Select an action At+1 ⇠ b(·|St+1)
                    A = e_greedy(Q, policy, S, goal)
                    
                    # store an action At+1 ⇠ b(·|St+1)
                    store.append((S, A, 0))
                    
                    # select and store sigma 
                    if sampling == False: sigma.append(1)
                    
                    else: sigma.append(0)
                        
                    if sigma[-1] == 1: sampling = True
                    
                    else: sampling = False
                    
                    # store pi(At+1|St+1)/b(At+1|St+1)
                    rho.append(pi(Q, policy, S)[A]/b(Q, policy, S)[A])
                    
            # tau <-- t - n +1
            tau = t - n + 1
            
            # If tau >=  0:
            if tau >= 0:
                
                # If t + 1 >= T: 
                if t+1 >= T: 
                    
                    # G <-- Q(St+1, At+1)
                    G = Q[store[t+1][0]][policy[store[t+1][0]].index(store[t+1][1])]
                    
                # Loop for k = min(t, T  1) down through tau + 1:
                for k in range(min(t+1, T), tau+1, 1):

                    # k at T
                    if k == T:
                        
                        # G <-- RT 
                        G = store[T][2]
                        
                    else:
                        # V <--  SUM a pi(a|Sk)*Q(Sk,a)
                        V += pi(Q, policy, store[k][0])[policy[store[k][0]].index(store[k][1])] * Q[store[k][0]][policy[store[k][0]].index(store[k][1])]
                        
                        # G <-- Rk + gamma *SUM a pi(a|St+1)*Q(St+1,a)
                        G = store[k][2] + (sigma[k]*rho[k] + gamma*(1 - sigma[k])*pi(Q, policy, store[k][0])[store[k][1]]) * (G - Q[store[k][0]][policy[store[k][0]].index(store[k][1])]) + gamma*V

                # Q(Stau, Atau) <-- Q(Stau , Atau) + a*p*[G - Q(Stau, Atau)]
                Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] = Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])] + alpha*(G - Q[store[tau][0]][policy[store[tau][0]].index(store[tau][1])]) 
                
            # Until t' = T - 1
            t += 1
            
    # Q = q*
    return Q

In [216]:
import pprint

# create terminal printer instance
pp = pprint.PrettyPrinter(width=160, compact=True)

# n-step boostrapping algorithms
print("n_step_sarsa")
pp.pprint(n_step_sarsa(alpha, n, episodes, start, goal))
print()
print("n_step_off_policy_sarsa")
pp.pprint(n_step_off_policy_sarsa(alpha, n, episodes, start, goal))
print()
print("n_step_per_decision")
pp.pprint(n_step_per_decision(alpha, n, episodes, start, goal))
print()
print("n_step_tree")
pp.pprint(n_step_tree(alpha, n, episodes, start, goal))
print()
print("n_step_qsigma")
pp.pprint(n_step_qsigma(alpha, gamma, n, episodes, start, goal))

n_step_sarsa
{(0, 0): [-58.095084186175264, 0, 0, -0.5779235664798654],
 (0, 1): [-66.83049827114556, 0, 0, -1.0986301265423835],
 (0, 2): [-64.83211873129659, 0, -0.5424521572479947, -0.9715730294672039],
 (0, 3): [-53.98692989088134, 0, -156.797718905483, -0.14273805764160638],
 (0, 4): [-63.57234081850753, 0, -14.265528265344017, -1.3055771011442097],
 (0, 5): [-65.41339697426444, 0, -0.13614290586634542, -1.210508849874543],
 (0, 6): [-45.543673200408904, 0, -56.18222855915444, -0.504035916439755],
 (0, 7): [-30.61717883282239, 0, -45.93827446922617, -1.1752419104979768],
 (0, 8): [-34.59734422726361, 0, -52.30079399023459, -0.12902893580512387],
 (0, 9): [-12.300303378720542, 0, -36.71265181206305, 0],
 (1, 0): [-64.39259705985083, -4.765144617382513, 0, -4.990375168327162],
 (1, 1): [-77.08336837880687, -3.939173120590491, -2.490075488345636, -0.9980976767944867],
 (1, 2): [-68.29910112779658, -10.40244862180167, -1.3150715900604346, -2.545214862631349],
 (1, 3): [-63.15469201278

KeyboardInterrupt: 